In [1]:
from keras.datasets import cifar10

In [2]:
# データ読み込み
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 64s 0us/step


In [3]:
print("x_train.shape:", x_train.shape)
print("x_test.shape:", x_test.shape)
print("y_train.shape:", y_train.shape)
print("y_test.shape:", y_test.shape)

x_train.shape: (50000, 32, 32, 3)
x_test.shape: (10000, 32, 32, 3)
y_train.shape: (50000, 1)
y_test.shape: (10000, 1)


In [5]:
# 学習データ加工

# 型変換
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# スケール変換
x_train /= 255.0
x_test /= 255.0

In [6]:
# 正解データ加工

from keras.utils import np_utils

# 数値を one-hot の形に変更
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [7]:
from keras.models import Sequential

# モデルの生成
model = Sequential()

In [8]:
# 畳み込み層の追加

from keras.layers import Conv2D

model.add(Conv2D(
    # チャンネル数(特徴マップの数)
    filters=32,
    # 入力層
    input_shape=(32, 32, 3),
    # カーネル(入力データにマスクする行列)のサイズ
    kernel_size=(3, 3),
    # ストライド幅(１マスずつ)
    strides=(1, 1),
    # ゼロパディングを利用して入力と出力のサイズ(縦横幅)を等しくする
    padding="same",
    # 活性化関数に Relu関数を指定
    activation="relu"
))

model.add(Conv2D(
    # チャンネル数(特徴マップの数)
    filters=32,
    # カーネル(入力データにマスクする行列)のサイズ
    kernel_size=(3, 3),
    # ストライド幅(１マスずつ)
    strides=(1, 1),
    # ゼロパディングを利用して入力と出力のサイズ(縦横幅)を等しくする
    padding="same",
    # 活性化関数に Relu関数を指定
    activation="relu"
))

In [9]:
# プーリング層の追加

from keras.layers import MaxPool2D

model.add(MaxPool2D(pool_size=(2, 2)))

In [10]:
# ドロップアウト

from keras.layers import Dropout

model.add(Dropout(0.25))

In [11]:
model.add(Conv2D(
    # チャンネル数(特徴マップの数)
    filters=64,
    # カーネル(入力データにマスクする行列)のサイズ
    kernel_size=(3, 3),
    # ストライド幅(１マスずつ)
    strides=(1, 1),
    # ゼロパディングを利用して入力と出力のサイズ(縦横幅)を等しくする
    padding="same",
    # 活性化関数に Relu関数を指定
    activation="relu"
))

model.add(Conv2D(
    # チャンネル数(特徴マップの数)
    filters=64,
    # カーネル(入力データにマスクする行列)のサイズ
    kernel_size=(3, 3),
    # ストライド幅(１マスずつ)
    strides=(1, 1),
    # ゼロパディングを利用して入力と出力のサイズ(縦横幅)を等しくする
    padding="same",
    # 活性化関数に Relu関数を指定
    activation="relu"
))

model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [12]:
# データ数, 縦, 横, チャンネル数
model.output_shape

(None, 8, 8, 64)

In [13]:
# 全結合層では ２次元テンソルしか入力出来ないので変換

from keras.layers import Flatten

model.add(Flatten())
model.output_shape

(None, 4096)

In [14]:
# 全結合層

from keras.layers import Dense

# 出力サイズ 512
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.5))

# 出力層, サイズ 10
model.add(Dense(units=10, activation="softmax"))

In [15]:
from keras.callbacks import TensorBoard

LOG_DIR = "./logs"
tsb = TensorBoard(log_dir=LOG_DIR)

In [16]:
# 学習プロセス設定
model.compile(
    # 勾配の算出にAdam
    optimizer="adam",
    # 損失関数にクロスエントロピー誤差
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [17]:
# 学習の実行
histroy_model = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=2,
    validation_split=0.2,
    callbacks=[tsb]
)

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 335s 8ms/step - loss: 1.5373 - acc: 0.4392 - val_loss: 1.1524 - val_acc: 0.5954
Epoch 2/2
40000/40000 [==============================] - 418s 10ms/step - loss: 1.1309 - acc: 0.5967 - val_loss: 0.9225 - val_acc: 0.6772
